In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import CelebA
from torchvision.transforms.v2 import (
    Compose,
    Resize,
    ColorJitter,
    RandomHorizontalFlip,
    ToImage,
    ToDtype,
)
from tqdm import tqdm
import os

# --- Config ---
DTYPE = torch.float32
BATCH_SIZE = 64
NUM_EPOCHS = 50
PATIENCE = 3
INPUT_SHAPE = (3, 64, 64)
SAVE_PATH = "./checkpoints/best_fcnn_model.pt"
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Transforms ---
train_transform = Compose([
    Resize((64, 64)),
    ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    RandomHorizontalFlip(),
    ToImage(),
    ToDtype(dtype=DTYPE, scale=True),
])

normal_transform = Compose([
    Resize((64, 64)),
    ToImage(),
    ToDtype(DTYPE, scale=True),
])

# --- Dataset path ---
data_path = "C:/Users/manju/Desktop/ECSE 552/Project"  # update if needed

# --- Datasets & Loaders ---
train_set = CelebA(data_path, split="train", target_type="attr", transform=train_transform, download=False)
val_set = CelebA(data_path, split="valid", target_type="attr", transform=normal_transform, download=False)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:

# --- Model ---
class FCNN(nn.Module):
    def __init__(self, input_shape=(3, 64, 64), num_classes=40):
        super(FCNN, self).__init__()
        c, h, w = input_shape
        input_dim = c * h * w

        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            nn.Sigmoid()  # for multi-label classification
        )

    def forward(self, x):
        return self.net(x)

# --- Initialize model on GPU ---
model = FCNN(input_shape=INPUT_SHAPE).to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [1]:

# --- Training Helpers ---
def transform_batch_labels(batch):
    images, labels = batch
    images = images.to(device, non_blocking=True)
    labels = labels.to(device, dtype=torch.float32, non_blocking=True)
    return images, labels

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for batch in tqdm(loader, desc="Training"):
        images, labels = transform_batch_labels(batch)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            images, labels = transform_batch_labels(batch)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    return running_loss / len(loader)

# --- Training with Early Stopping ---
best_val_loss = float("inf")
epochs_no_improve = 0

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, val_loader, criterion)
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved to {SAVE_PATH}")
    else:
        epochs_no_improve += 1
        print(f"⏸️ No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= PATIENCE:
            print("🛑 Early stopping triggered.")
            break

# --- Load best model ---
model.load_state_dict(torch.load(SAVE_PATH))
print("🔁 Loaded best model weights from disk.")


In [11]:
import numpy as np
def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            probs = torch.sigmoid(logits)
            pred_binary = (probs > 0.5).float()

            all_preds.append(pred_binary.cpu().numpy())
            all_labels.append(y.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    return all_preds, all_labels

test_set = CelebA(data_path, split="test", target_type="attr", transform=normal_transform, download=False)
test_loader = DataLoader(test_set, batch_size=32)
all_preds, all_labels = evaluate(model, test_loader, device)

In [13]:
import numpy as np
import matplotlib.pyplot as plt

# Ensure preds and labels are numpy arrays
# all_preds: binary predictions (0/1), shape: [N, 40]
# all_labels: ground truth labels (0/1), shape: [N, 40]

# Element-wise correctness
correct_matrix = (all_preds == all_labels).astype(np.float32)

# Per-attribute accuracy
attribute_accuracy = correct_matrix.mean(axis=0)  # shape: [40]

# Overall accuracy (average across all attributes and samples)
overall_accuracy = correct_matrix.mean()

print(f"Overall accuracy: {overall_accuracy:.4f}")


In [10]:
# Calculate per-attribute accuracy
correct_matrix = (all_preds == all_labels).astype(np.float32)
attribute_accuracy = correct_matrix.mean(axis=0)  # shape: (40,)

# Print each attribute's accuracy
for i, attr in enumerate(ATTRIBUTES):
    print(f"{attr:25s}: {attribute_accuracy[i]:.4f}")


In [15]:
import pandas as pd
import numpy as np

def save_preds_labels_to_csv(all_preds, all_labels, filename="results.csv", attributes=None):
    """
    Save predictions and labels to a CSV file side-by-side.
    If attribute names are given, use them as column headers.
    """
    # Make sure arrays are numpy
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Prefix column names
    if attributes is None:
        attributes = [f"attr_{i}" for i in range(all_preds.shape[1])]

    pred_cols = [f"pred_{attr}" for attr in attributes]
    label_cols = [f"true_{attr}" for attr in attributes]

    df_preds = pd.DataFrame(all_preds, columns=pred_cols)
    df_labels = pd.DataFrame(all_labels, columns=label_cols)

    df = pd.concat([df_preds, df_labels], axis=1)
    df.to_csv(filename, index=False)
    print(f"✅ Saved predictions and labels to {filename}")



save_preds_labels_to_csv(all_preds, all_labels, filename="fcnn_test_results.csv", attributes=ATTRIBUTES)
